In [ ]:
import numpy as np
import numpy.linalg as la
import math
import timeit

#Setting up the parameters and initial guess
A = np.array([[2,-1,0,0],[-1,20000,-1,0],[-0,-1,200,-1],[0,0,-1,2]])
b = np.array([1,2,3,4])
x_init = [0.1,0.1,0.1,0.1]

#Inverse Jacobi preconditioning matrix
M_inv_elem = []
for i in range(len(A)):
    M_inv_elem.append(1/A[i][i])
M_inv = np.diag(M_inv_elem)

#Optimum relaxation parameter for SOR
U = np.array([[0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [0,0,0,0]])
L = np.array([[0,0,0,0],
              [1,0,0,0],
              [0,1,0,0],
              [0,0,1,0]])
MJ = np.matmul(M_inv, np.add(L,U))
(eigvalMJ,eigvecMJ) =la.eig(MJ)
w_opt = 2/(1+ math.sqrt(1-max(eigvalMJ)**2))

#CG method
def CG(A, b, x_init, epsi):
    x = x_init
    r = b - np.matmul(A,x)
    n = 0
    while any (abs(y)>epsi for y in r):
        r_prime = np.matmul(A,r)
        stepsize = np.dot(r,r)/np.dot(r,r_prime)
        x = x + stepsize*r
        r = r - stepsize*r_prime
        n+=1
    return [x,n]

#Preconditioned CG
def PCG(A, b, M_inv, x_init, epsi):
    E_inv = np.sqrt(M_inv)
    A = np.matmul(E_inv,A)
    A = np.matmul(A,E_inv)
    b = np.matmul(E_inv,b)
    [x,n] = CG(A,b,x_init,epsi)
    x = np.matmul(E_inv,x)
    return [x,n]

#OSR
def OSR(A, b, x_init, w, epsi):
    x = x_init
    err = b-np.matmul(A,x)
    n = 0
    while any (abs(y)>epsi for y in err):
        for i in range(len(x_init)):
            GS = 0 #Gauss-siedel term
            for j in range(len(x_init)):
                if j!=i:
                    GS-=(A[i][j]/A[i][i])*x[j]
            GS += b[i]/A[i][i]
            x[i] = (1-w)*x[i]+w*GS
        n+=1
        err = b-np.matmul(A,x)
    return [x,n]

#Wrapping for timing purpose
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

wrapped1 = wrapper(CG, A, b, x_init, 10**(-8))
wrapped2 = wrapper(PCG, A, b, M_inv, x_init, 10**(-8))
wrapped3 = wrapper(OSR, A, b, x_init, w_opt, 10**(-8))
[CG_sol, CG_iter] = CG(A,b,x_init,10**(-8))
[PCG_sol, PCG_iter] = PCG(A,b,M_inv,x_init,10**(-8))
[OSR_sol, OSR_iter] = OSR(A, b, x_init, w_opt, 10**(-8))

print("#CG method")
print("x = "+str(CG_sol.tolist()))
print("Converges after "+str(CG_iter)+" iterations")
print("Computing time = "+str(timeit.timeit(wrapped1, number=100))+" seconds") #Time elapsed after 100 operations

print("#PCG method")
print("x = "+str(PCG_sol.tolist()))
print("Converges after "+str(PCG_iter)+" iterations")
print("Computing time  = "+str(timeit.timeit(wrapped2, number=100))+" seconds")


print("#OSR method")
print("x = "+str(OSR_sol))
print("Converges after "+str(OSR_iter)+" iterations")
print("Computing time  = "+str(timeit.timeit(wrapped3, number=100))+" seconds")

In [ ]:
x_init = [0,0,0,0]

def OSR(A, b, x_init, w, k):
    x = x_init
    for l in range(k):
        for i in range(len(x_init)):
            GS = 0 #Gauss-siedel term
            for j in range(len(x_init)):
                if j!=i:
                    GS-=(A[i][j]/A[i][i])*x[j]
            GS += b[i]/A[i][i]
            x[i] = (1-w)*x[i]+w*GS
    return x

OSR(A, b, x_init, w_opt, 100)